In [ ]:
#모듈
import math
import pandas as pd
import numpy as np

try:
    import cufflinks as cf
except:
    !pip install cufflinks
    import cufflinks as cf

cf.go_offline()

from sklearn.ensemble import RandomForestRegressor

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
# 데이터 다루기
pd.set_option("Display.max_columns",200)
import re
import time

# 데이터 시각화
import seaborn as sns

# 사용할 머신러닝 모델들 - RandomForest, AdaBoost, GradientBoost, ExtraTrees
import sklearn
from sklearn.metrics import log_loss
from sklearn.svm import SVC

In [ ]:
#명목형 변수 더미화
def dummify_category_cols(df,cols):
    dummies = []
    new_df = df.copy()

    for idx in range(len(cols)):
        new_df = new_df.join(pd.get_dummies(total[cols[idx]], prefix = cols[idx]))
        new_df = new_df.drop(cols[idx], axis =1)
    return new_df

In [ ]:
#숫자형 변수 범주화
def dummify_number_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: int(x)//num * num)
    return new_df


In [ ]:
#명목형 변수의 연속형 변환 - 변수의 차원을 낮추기 위하여
def num_category(df, col, chg) :
    new_df = df.copy()
    set = {}
    
    for idx in range(len(chg)) :
        set[chg[idx]] = idx
        
    new_df[col] = new_df[col].map(set)
    
    return new_df        

In [ ]:
#숫자형 변수 일정 이상은 묶어버리기
def number_group_cols(df, cols, nums) :
    new_df = df.copy() 
    
    for col, num in zip(cols, nums) :
        new_df[col] =new_df[col].map(lambda x: num if x> num else x )
    return new_df


In [ ]:
#RMS 구하기
def RMSE(y_true,y_pred):
    
    squaredErrors = (y_pred - y_true)**2
    MSE = squaredErrors.mean()
    
    return math.sqrt(MSE)

In [ ]:
#본격적으로 시작

In [ ]:
training = pd.read_csv("house_price.csv")
test = pd.read_csv("test.csv")

#training = training[np.isfinite(training["GarageYrBlt"])]

In [ ]:
total = pd.concat([training, test]).reset_index()
del total["index"]

In [ ]:
total.head(2)

In [ ]:
total.columns

In [ ]:
#선택된 변수
category_selected = ["MSSubClass", "MSZoning", "Street" ,"LandContour", "LotConfig", 
            "Neighborhood", "BldgType", "HouseStyle", "OverallQual", 
            "MasVnrType", "ExterQual", "ExterCond",
            "Foundation","BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC", "CentralAir",
            "Electrical", "BsmtFullBath", "FullBath", "HalfBath", "KitchenQual",
            "TotRmsAbvGrd", "Functional", "Fireplaces", "FireplaceQu", "GarageType",
            "GarageCars", "PavedDrive", "SaleCondition", "GarageFinish"]

extra_selected = [ "GarageCars", "Electrical" ,"OverallQual","OverallCond","ExterQual", "ExterCond",  "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1", "HeatingQC","KitchenQual","FireplaceQu", "Functional"]

number_selected = ["GarageYrBlt","YearBuilt", "YearRemodAdd", "BsmtFinSF1", "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "GrLivArea", 'GarageArea']

category_selected = [x for x in category_selected if x not in extra_selected]

total_selected = category_selected + number_selected + extra_selected

selected_with_SalePrice = total_selected +["SalePrice"]

In [ ]:
new_total = total[selected_with_SalePrice].copy()
new_total.head(2)

### 임시 전처리

In [ ]:
new_total[number_selected].isnull().sum()

In [ ]:
new_total["YearBuilt"] = new_total["YearBuilt"].fillna(new_total.YearBuilt.mean())
new_total["BsmtFinSF1"] = new_total["BsmtFinSF1"].fillna(new_total.BsmtFinSF1.mean())
new_total["BsmtUnfSF"] = new_total["BsmtUnfSF"].fillna(new_total.BsmtUnfSF.mean())
new_total["TotalBsmtSF"] = new_total["TotalBsmtSF"].fillna(new_total.TotalBsmtSF.mean())
new_total["GarageArea"] = new_total["GarageArea"].fillna(new_total.GarageArea.mean())
new_total["GarageYrBlt"] = new_total["GarageYrBlt"].fillna(new_total.GarageYrBlt.mean())
new_total["Electrical"] = new_total["Electrical"].fillna("SBrkr")
new_total = new_total.fillna("404")
new_total.isnull().sum()

In [ ]:
new_total = num_category(new_total, "ExterQual", ["Po", "Fa", "TA", "Gd", "Ex"])
new_total = num_category(new_total, "ExterCond", ["Po", "Fa", "TA", "Gd", "Ex"])
new_total = num_category(new_total, "BsmtQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])
new_total = num_category(new_total, "BsmtCond", ["404", "Po", "Fa", "TA", "Gd", "Ex"])
new_total = num_category(new_total, "BsmtExposure", ["404", "No", "Fa", "Mn", "Av", "Gd"])
new_total = num_category(new_total, "BsmtFinType1", ["404", "Unf", "LwQ", "Rec", "BLQ", "ALQ", "GLQ"])
new_total = num_category(new_total, "HeatingQC", ["404", "Po", "Fa", "TA", "Gd", "Ex"])
new_total = num_category(new_total, "KitchenQual", ["404", "Po", "Fa", "TA", "Gd", "Ex"])
new_total = num_category(new_total, "FireplaceQu", ["404", "Po", "Fa", "TA", "Gd", "Ex"])
new_total = num_category(new_total, "Electrical", ["Mix", "FuseP", "FuseF", "FuseA", "SBrkr"])
new_total = num_category(new_total, "Functional", ["404", "Sal", "Sev", "Maj2", "Maj1", "Mod", "Min2", "Min1", "Typ"])
new_total.Functional.unique()

In [ ]:
new_total["Functional"] = total["Functional"].map(lambda x : 1 if x==8 else 0)
new_total["LotConfig"] = new_total.LotConfig.map(lambda x : 1 if x == "CulDSac" else 0)
new_total["FullBath"] = new_total.FullBath.map(lambda x : 1 if x <=2 else x-1)
new_total["OverallCond"] = new_total.OverallCond.map(lambda x : 1 if x <=4 else 2)
new_total["BldgType"] = new_total.BldgType.map(lambda x : 1 if x =="1Fam" else 2)
new_total = number_group_cols(new_total, ["TotRmsAbvGrd"], [13])
new_total = number_group_cols(new_total, ["HalfBath"], [1])
new_total = number_group_cols(new_total, ["FullBath"], [2])

In [ ]:
new_total.head(2)

<br><br>

In [ ]:
#히트맵으로 확인
training = training = new_total.iloc[:len(training)]

n= len(selected_with_SalePrice)
colormap = plt.cm.viridis
plt.figure(figsize=(n,n))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(training.corr(),
            linewidths=0.1,
            vmax=1.0, 
            square=True, 
            cmap=colormap, 
            linecolor='white', 
            annot=True)

In [ ]:
n= len(selected_with_SalePrice)
colormap = plt.cm.viridis
plt.figure(figsize=(n,n))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(training.corr(),
            linewidths=0.1,
            vmax=1.0, 
            square=True, 
            cmap=colormap, 
            linecolor='white', 
            annot=True)

<br><br>

In [ ]:
training['is_train'] = np.random.uniform(0, 1, len(training)) <= .75

training_set = training[training['is_train']==True]
test_set = training[training['is_train']==False]

In [ ]:
N = 4500
forest = RandomForestRegressor(n_estimators=N, n_jobs=2)

feature = new_total2.columns.drop("SalePrice")
target = ["SalePrice"]

forest.fit(training_set[feature], training_set[target])
forest.predict(test_set[feature])
forest.score(test_set[feature], test_set[target])
y_pred=forest.predict(test_set[feature])

rmse = RMSE(y_true,y_pred)
print(rmse)

In [ ]:
importance = pd.DataFrame(forest.feature_importances_, index = training_set[feature].columns)
(
    importance.assign(header = lambda x: x.index.str.split("_").str[0])
                  .groupby("header")[0]
                  .sum()
                  .iplot(kind="bar")
)

<br><br>

In [ ]:
# 최적화할 파라미터 N
#best_score = 0
#for N in range(2000,6000,500):
#    tmp = RandomForestRegressor(n_estimators=N)
#    tmp.fit(training_set[feature], training_set[target])
#    score = tmp.score(test_set[feature],test_set[target])
#    
    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
#    if score>best_score:
#        best_score = score
#        best_N = N
#        best_rf_model = tmp
        
#print("최적화 n_estimator 값 : {}".format(best_N))
#print("예측률 : {}".format(best_score))

In [ ]:
#best_score = 0
#for N in range(13,21):
#    tmp = RandomForestRegressor(n_estimators=4500, max_depth = N)
#    tmp.fit(training_set[feature], training_set[target])
#    score = tmp.score(test_set[feature],test_set[target])
#    
#    # 만약 기존의 모델보다 더 예측률이 높다면, 최신화합니다
#    if score>best_score:
#        best_score = score
#        best_N = N
#        best_rf_model = tmp
        
#print("최적화 max_depth 값 : {}".format(best_N))
#print("예측률 : {}".format(best_score))

In [ ]:
forest.fit(training[feature], training[target])
prediction =forest.predict(test[feature])
submission = pd.DataFrame({'Id':list(test.index+1),'SalePrice':prediction})
submission.to_csv("Submission10.csv")